In [3]:
import os
import requests
import re
import json
import urllib.request
from bs4 import BeautifulSoup

# Define headers and search parameters
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36"
}

params = {
    "q": "bệnh huyết trùng trên lá cà phê",  # Search query
    "tbm": "isch",                         # Image search
    "hl": "vn",                            # Language
    "gl": "vn",                            # Country
    "ijn": "0"                             # Page number
}

def create_folder(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

def download_images(images, folder_path, max_images=100):
    for index, img_url in enumerate(images[:max_images], start=1):
        try:
            print(f"Downloading image {index}/{max_images}...")
            image_path = os.path.join(folder_path, f"youtube_{index}.jpg")
            urllib.request.urlretrieve(img_url, image_path)
        except Exception as e:
            print(f"Failed to download image {index}: {e}")

def scrape_google_images(params, headers, folder_path, max_images=100):
    response = requests.get("https://www.bing.com/search", params=params, headers=headers, timeout=30)
    soup = BeautifulSoup(response.text, "html.parser")

    # Extract image URLs using regex
    scripts = soup.find_all("script")
    image_data = "".join(re.findall(r"AF_initDataCallback\(([^<]+)\);", str(scripts)))
    image_data_json = json.loads(json.dumps(image_data))  # Ensure JSON compatibility

    # Full-resolution image URLs
    image_urls = re.findall(r"(https?://[^,]+?\.(?:jpg|png|gif))", image_data_json)
    
    # Create folder and download images
    create_folder(folder_path)
    download_images(image_urls, folder_path, max_images)

if __name__ == "__main__":
    output_folder = "nematode_disease_in_coffee_leaf_images"
    scrape_google_images(params, headers, output_folder, max_images=100)
